In [ ]:
!pip install streamlit


In [ ]:
!pip install fbprophet

In [ ]:
!pip install yfinance

In [ ]:
import streamlit as st
from datetime import date
import yfinance as yf
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from plotly import graph_objs as go

In [ ]:
# We want our data from 2015
START = "2015-01-01"
TODAY =  date.today().strftime("%Y-%m-%d")

In [ ]:
st.title("Stock Prediction App")

stocks = ("AAPL","GOOG","MSFT","GME")

selected_stocks = st.selectbox("Select dataset for prediction",stocks)
n_years = st.slider("Years of prediction:",1,4)
periods = n_years*365

2021-12-30 09:04:23.972 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
%%writefile app.py
from datetime import date
import yfinance as yf
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from plotly import graph_objs as go
import streamlit as st
START = "2015-01-01"
TODAY =  date.today().strftime("%Y-%m-%d")

st.title("Stock Prediction App")

stocks = ("FB2A.BE","AAPL","GOOG","AMZN","NFLX")

selected_stocks = st.selectbox("Select dataset for prediction",stocks)
n_years = st.slider("Years of prediction:",1,4)
period = n_years*365

#loading data
@st.cache                   #Due to this once data is downloaded you dont need to download it again
def  load_data(ticker):
  data = yf.download(ticker,START,TODAY)
  data.reset_index(inplace=True) 
  return data

data_load_state = st.text("Load data...")  
data = load_data(selected_stocks)

st.subheader('Raw Data')
st.write(data.tail())

def plot_raw_data():
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=data['Date'], y = data['Open'],name = 'stock_open'))
  fig.add_trace(go.Scatter(x=data['Date'], y = data['Close'],name = 'stock_close'))
  fig.layout.update(title_text="Time Series Data" , xaxis_rangeslider_visible = True)
  st.plotly_chart(fig)

plot_raw_data()  

#Forcasting 
df_train = data[['Date','Close']]
df_train = df_train.rename(columns={"Date":"ds","Close":"y"})  #nameing according to fb prophet documentation

m = Prophet()
m.fit(df_train) # Training the model with Date and Close
future = m.make_future_dataframe(periods=period)
forcast = m.predict(future)

st.header('Forcasted Data')
st.write(forcast.tail())   #This .tail will give us the last ten rows

fig1 = plot_plotly(m,forcast)
st.plotly_chart(fig1)

st.write('forcast components')
fig2 = m.plot_components(forcast)
st.write(fig2)









Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.199.49.134:8501

2021-12-30 09:14:05.192 NumExpr defaulting to 2 threads.
[*********************100%***********************]  1 of 1 completed
  Stopping...
2021-12-30 09:14:07.194 Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Initial log joint probability = -62.3304
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       5242.81    0.00436866       829.219      0.9167      0.9167      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       5328.88     0.0233872       2209.13      0.4199      0.4199      239   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       5368.39      0.049696       1404.52      0.9385      0.9385      348   
    Iter      log prob        ||d

KeyboardInterrupt: ignored